In [2]:
import pandas as pd
import requests
from datetime import datetime
import os

os.makedirs('./data/raw', exist_ok=True)
os.makedirs('./data/processed', exist_ok=True)

print("✅ Setup completed")



✅ Setup completed


In [3]:
# Cell 2: URL 접근 테스트
owid_url = "https://catalog.ourworldindata.org/garden/covid/latest/compact/compact.csv"

# 먼저 URL이 접근 가능한지 확인
try:
    response = requests.head(owid_url, timeout=10)
    print(f"✅ URL 접근 성공: {response.status_code}")
    print(f"📊 Content-Length: {response.headers.get('content-length', 'Unknown')} bytes")
except Exception as e:
    print(f"❌ URL 접근 실패: {e}")
    

✅ URL 접근 성공: 200
📊 Content-Length: 157527890 bytes


In [8]:
# Cell 3: 소량 데이터로 먼저 테스트
print("📥 Downloading sample data...")

# 처음 몇 줄만 읽어서 구조 확인
sample_df = pd.read_csv(owid_url, nrows=1000)
# Cell 3-1: 실제 컬럼 구조 확인
print("📋 Actual columns in the dataset:")
print("=" * 50)
for i, col in enumerate(sample_df.columns):
    print(f"  {i+1:2d}. {col}")

print(f"\n📊 Total columns: {len(sample_df.columns)}")

# Cell 3-2: 국가/지역 관련 컬럼 찾기
print("🔍 Looking for country/location related columns:")
location_columns = [col for col in sample_df.columns if any(keyword in col.lower() for keyword in ['country', 'location', 'nation', 'region'])]

if location_columns:
    print("✅ Found location-related columns:")
    for col in location_columns:
        print(f"  - {col}")
        print(f"    Sample values: {sample_df[col].unique()[:5]}")
else:
    print("❌ No obvious location columns found")
    print("🔍 Let's check all columns for potential country data:")
    
    # 각 컬럼의 샘플 값들 확인
    for col in sample_df.columns[:10]:  # 처음 10개 컬럼만 확인
        unique_vals = sample_df[col].unique()
        if len(unique_vals) < 50:  # 고유값이 50개 미만인 컬럼들 확인
            print(f"  {col}: {unique_vals[:5]}")

# Cell 3-3: 'country' 컬럼이 있는지 확인하고 한국 데이터 찾기
# 일반적으로 OWID에서는 'country' 컬럼을 사용할 가능성이 높음
possible_country_columns = ['country', 'location', 'entity', 'region', 'nation']

country_column = None
for col in possible_country_columns:
    if col in sample_df.columns:
        country_column = col
        print(f"✅ Found country column: '{col}'")
        break

if country_column:
    # 한국 관련 데이터 찾기
    korea_variations = ['South Korea', 'Korea', 'Republic of Korea', 'KOR', 'Korea, South']
    
    print(f"\n🔍 Looking for Korea data in '{country_column}' column:")
    unique_countries = sample_df[country_column].unique()
    
    korea_matches = [country for country in unique_countries if any(korea_name in str(country) for korea_name in korea_variations)]
    
    if korea_matches:
        print(f"✅ Found Korea data: {korea_matches}")
        korea_data = sample_df[sample_df[country_column].isin(korea_matches)]
        print(f"🇰🇷 Korea rows in sample: {len(korea_data)}")
    else:
        print("❌ No Korea data found. Available countries:")
        print(unique_countries[:20])  # 처음 20개 국가만 표시
else:
    print("❌ No country column found. Let's examine the data structure more carefully.")

📥 Downloading sample data...
📋 Actual columns in the dataset:
   1. country
   2. date
   3. total_cases
   4. new_cases
   5. new_cases_smoothed
   6. total_cases_per_million
   7. new_cases_per_million
   8. new_cases_smoothed_per_million
   9. total_deaths
  10. new_deaths
  11. new_deaths_smoothed
  12. total_deaths_per_million
  13. new_deaths_per_million
  14. new_deaths_smoothed_per_million
  15. excess_mortality
  16. excess_mortality_cumulative
  17. excess_mortality_cumulative_absolute
  18. excess_mortality_cumulative_per_million
  19. hosp_patients
  20. hosp_patients_per_million
  21. weekly_hosp_admissions
  22. weekly_hosp_admissions_per_million
  23. icu_patients
  24. icu_patients_per_million
  25. weekly_icu_admissions
  26. weekly_icu_admissions_per_million
  27. stringency_index
  28. reproduction_rate
  29. total_tests
  30. new_tests
  31. total_tests_per_thousand
  32. new_tests_per_thousand
  33. new_tests_smoothed
  34. new_tests_smoothed_per_thousand
  35. pos

In [9]:
# Cell 4: 더 많은 데이터로 한국 찾기
print("🔍 Looking for Korea in larger dataset...")

# 단계적으로 데이터 크기를 늘려가며 한국 데이터 찾기
sample_sizes = [5000, 10000, 20000, 50000]

korea_found = False
for sample_size in sample_sizes:
    print(f"\n📥 Checking first {sample_size:,} rows...")
    
    try:
        larger_sample = pd.read_csv(owid_url, nrows=sample_size)
        unique_countries = larger_sample['country'].unique()
        
        print(f"✅ Found {len(unique_countries)} countries")
        print(f"📋 Countries from {unique_countries[0]} to {unique_countries[-1]}")
        
        # 한국 찾기
        korea_variations = ['South Korea', 'Korea']
        korea_matches = [country for country in unique_countries 
                        if any(korea_name in str(country) for korea_name in korea_variations)]
        
        if korea_matches:
            print(f"🎉 Found Korea: {korea_matches}")
            korea_data = larger_sample[larger_sample['country'].isin(korea_matches)]
            print(f"🇰🇷 Korea rows: {len(korea_data)}")
            print(f"📅 Date range: {korea_data['date'].min()} ~ {korea_data['date'].max()}")
            korea_found = True
            break
        else:
            print(f"❌ Korea not found yet")
            
    except Exception as e:
        print(f"❌ Error with {sample_size} rows: {e}")

if not korea_found:
    print("\n🔍 Let's check the full dataset (may take longer)...")

🔍 Looking for Korea in larger dataset...

📥 Checking first 5,000 rows...
✅ Found 3 countries
📋 Countries from Afghanistan to Albania
❌ Korea not found yet

📥 Checking first 10,000 rows...
✅ Found 5 countries
📋 Countries from Afghanistan to American Samoa
❌ Korea not found yet

📥 Checking first 20,000 rows...
✅ Found 10 countries
📋 Countries from Afghanistan to Argentina
❌ Korea not found yet

📥 Checking first 50,000 rows...
✅ Found 25 countries
📋 Countries from Afghanistan to Benin
❌ Korea not found yet

🔍 Let's check the full dataset (may take longer)...


In [11]:
# Cell 5-alternative: 더 효율적인 방법으로 한국 찾기
print("🔍 Alternative approach: Search for Korea more efficiently...")

try:
    # 청크 단위로 읽어서 한국 찾기
    chunk_size = 10000
    korea_found = False
    chunks_checked = 0
    
    print(f"📥 Reading data in chunks of {chunk_size:,} rows...")
    
    for chunk in pd.read_csv(owid_url, chunksize=chunk_size):
        chunks_checked += 1
        
        # 현재 청크의 국가들 확인
        countries_in_chunk = chunk['country'].unique()
        
        print(f"Chunk {chunks_checked}: {countries_in_chunk[0]} to {countries_in_chunk[-1]} ({len(countries_in_chunk)} countries)")
        
        # 한국 찾기
        korea_matches = [country for country in countries_in_chunk 
                        if 'south korea' in str(country).lower()]
        
        if korea_matches:
            print(f"🎉 Found Korea in chunk {chunks_checked}: {korea_matches}")
            korea_data = chunk[chunk['country'].isin(korea_matches)]
            print(f"🇰🇷 Korea rows in this chunk: {len(korea_data)}")
            korea_found = True
            break
            
        # 50개 청크까지 체크
        if chunks_checked >= 50:
            print("Checked 50 chunks, continuing to search...")
            
except Exception as e:
    print(f"❌ Chunk reading failed: {e}")

🔍 Alternative approach: Search for Korea more efficiently...
📥 Reading data in chunks of 10,000 rows...
Chunk 1: Afghanistan to American Samoa (5 countries)
Chunk 2: American Samoa to Argentina (6 countries)
Chunk 3: Argentina to Australia (6 countries)
Chunk 4: Australia to Bangladesh (6 countries)
Chunk 5: Bangladesh to Benin (6 countries)
Chunk 6: Benin to Bosnia and Herzegovina (6 countries)
Chunk 7: Bosnia and Herzegovina to Brunei (5 countries)
Chunk 8: Brunei to Cameroon (6 countries)
Chunk 9: Cameroon to Chad (6 countries)
Chunk 10: Chad to Congo (6 countries)
Chunk 11: Congo to Cuba (6 countries)
Chunk 12: Cuba to Denmark (6 countries)
Chunk 13: Denmark to Ecuador (6 countries)
Chunk 14: Ecuador to Eritrea (7 countries)
Chunk 15: Eritrea to European Union (27) (6 countries)
Chunk 16: European Union (27) to France (6 countries)
Chunk 17: France to Gambia (5 countries)
Chunk 18: Gambia to Greece (6 countries)
Chunk 19: Greece to Guatemala (6 countries)
Chunk 20: Guatemala to Hai

In [13]:
# Cell 6 (수정): 한국 데이터 확인 및 분석 - 경고 해결
print("🎉 Korea data found! Analyzing...")

korea_country_name = 'South Korea'  # 확인된 정확한 국가명
korea_found = True

# SettingWithCopyWarning 해결: .copy() 사용
korea_data = korea_data.copy()  # 명시적 복사본 생성

print(f"✅ Country name in dataset: '{korea_country_name}'")
print(f"🇰🇷 Total Korea rows found: {len(korea_data)}")

# 날짜 범위 확인
if 'date' in korea_data.columns:
    korea_data['date'] = pd.to_datetime(korea_data['date'])
    
    # 현재 날짜까지의 실제 데이터만 필터링
    today = datetime.now().date()
    actual_data = korea_data[korea_data['date'].dt.date <= today].copy()
    
    print(f"📅 Full date range: {korea_data['date'].min()} ~ {korea_data['date'].max()}")
    print(f"📅 Actual data range: {actual_data['date'].min()} ~ {actual_data['date'].max()}")
    print(f"📊 Total rows: {len(korea_data)} (including future dates)")
    print(f"📊 Actual data rows: {len(actual_data)} (up to today)")
    
    # 실제 데이터를 메인으로 사용
    korea_data = actual_data

# 주요 컬럼 데이터 가용성 확인
key_columns = [
    'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
    'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred',
    'new_cases_smoothed', 'reproduction_rate', 'stringency_index'
]

print(f"\n📋 Key data availability (actual data only):")
for col in key_columns:
    if col in korea_data.columns:
        non_null = korea_data[col].notna().sum()
        percentage = (non_null / len(korea_data)) * 100
        print(f"  ✅ {col}: {non_null}/{len(korea_data)} ({percentage:.1f}%)")
    else:
        print(f"  ❌ {col}: Not available")

# 실제 데이터의 최근 샘플 확인
print(f"\n📊 Recent actual data sample (last 5 rows):")
recent_cols = ['date', 'new_cases', 'total_cases', 'new_deaths', 'people_vaccinated_per_hundred']
print(korea_data[recent_cols].tail())

# 데이터 기간별 분석
print(f"\n📈 Data completeness by period:")
korea_data_copy = korea_data.copy()
korea_data_copy['year'] = korea_data_copy['date'].dt.year

for year in sorted(korea_data_copy['year'].unique()):
    year_data = korea_data_copy[korea_data_copy['year'] == year]
    cases_available = year_data['total_cases'].notna().sum()
    vaccine_available = year_data['people_vaccinated_per_hundred'].notna().sum()
    print(f"  {year}: Cases {cases_available}/{len(year_data)} ({cases_available/len(year_data)*100:.1f}%), "
          f"Vaccines {vaccine_available}/{len(year_data)} ({vaccine_available/len(year_data)*100:.1f}%)")

🎉 Korea data found! Analyzing...
✅ Country name in dataset: 'South Korea'
🇰🇷 Total Korea rows found: 2081
📅 Full date range: 2020-01-01 00:00:00 ~ 2025-12-28 00:00:00
📅 Actual data range: 2020-01-01 00:00:00 ~ 2025-09-21 00:00:00
📊 Total rows: 2081 (including future dates)
📊 Actual data rows: 2067 (up to today)

📋 Key data availability (actual data only):
  ✅ total_cases: 2060/2067 (99.7%)
  ✅ new_cases: 2060/2067 (99.7%)
  ✅ total_deaths: 2060/2067 (99.7%)
  ✅ new_deaths: 2059/2067 (99.6%)
  ✅ people_vaccinated_per_hundred: 655/2067 (31.7%)
  ✅ people_fully_vaccinated_per_hundred: 655/2067 (31.7%)
  ✅ new_cases_smoothed: 2055/2067 (99.4%)
  ✅ reproduction_rate: 1047/2067 (50.7%)
  ✅ stringency_index: 1096/2067 (53.0%)

📊 Recent actual data sample (last 5 rows):
             date  new_cases  total_cases  new_deaths  \
438727 2025-08-24        0.0   34571873.0         0.0   
438728 2025-08-31        NaN          NaN         NaN   
438729 2025-09-07        NaN          NaN         NaN   

In [19]:
# Cell 6-1: 데이터 컬럼 상세 분석
print("📋 COVID-19 Dataset Column Information")
print("=" * 60)

def analyze_column_info(data):
    """각 컬럼의 정보를 분석하고 설명"""
    
    # 컬럼 그룹별 분류 및 설명
    column_groups = {
        "📈 Cases & Deaths (확진자 & 사망자)": {
            'total_cases': '누적 확진자 수',
            'new_cases': '일일 신규 확진자 수',
            'new_cases_smoothed': '일일 신규 확진자 수 (7일 이동평균)',
            'total_cases_per_million': '인구 100만명 당 누적 확진자 수',
            'new_cases_per_million': '인구 100만명 당 일일 신규 확진자 수',
            'new_cases_smoothed_per_million': '인구 100만명 당 일일 신규 확진자 수 (7일 이동평균)',
            'total_deaths': '누적 사망자 수',
            'new_deaths': '일일 신규 사망자 수',
            'new_deaths_smoothed': '일일 신규 사망자 수 (7일 이동평균)',
            'total_deaths_per_million': '인구 100만명 당 누적 사망자 수',
            'new_deaths_per_million': '인구 100만명 당 일일 신규 사망자 수',
            'new_deaths_smoothed_per_million': '인구 100만명 당 일일 신규 사망자 수 (7일 이동평균)'
        },
        
        "💉 Vaccination (백신 접종)": {
            'total_vaccinations': '총 백신 접종 회수 (1차+2차+부스터 모두 포함)',
            'people_vaccinated': '1차 접종을 받은 사람 수',
            'people_fully_vaccinated': '완전 접종을 받은 사람 수 (2차까지)',
            'total_boosters': '부스터 접종 총 회수',
            'new_vaccinations': '일일 신규 백신 접종 회수',
            'new_vaccinations_smoothed': '일일 신규 백신 접종 회수 (7일 이동평균)',
            'total_vaccinations_per_hundred': '인구 100명 당 총 백신 접종 회수',
            'people_vaccinated_per_hundred': '인구 100명 당 1차 접종률 (%)',
            'people_fully_vaccinated_per_hundred': '인구 100명 당 완전 접종률 (%)',
            'total_boosters_per_hundred': '인구 100명 당 부스터 접종 회수',
            'new_vaccinations_smoothed_per_million': '인구 100만명 당 일일 신규 백신 접종 회수 (7일 이동평균)',
            'new_people_vaccinated_smoothed': '일일 신규 1차 접종자 수 (7일 이동평균)',
            'new_people_vaccinated_smoothed_per_hundred': '인구 100명 당 일일 신규 1차 접종률 (7일 이동평균)'
        },
        
        "🏥 Healthcare System (의료 시스템)": {
            'hosp_patients': '현재 입원 환자 수',
            'hosp_patients_per_million': '인구 100만명 당 현재 입원 환자 수',
            'weekly_hosp_admissions': '주간 신규 입원 환자 수',
            'weekly_hosp_admissions_per_million': '인구 100만명 당 주간 신규 입원 환자 수',
            'icu_patients': '현재 중환자실(ICU) 환자 수',
            'icu_patients_per_million': '인구 100만명 당 현재 중환자실 환자 수',
            'weekly_icu_admissions': '주간 신규 중환자실 입원 환자 수',
            'weekly_icu_admissions_per_million': '인구 100만명 당 주간 신규 중환자실 입원 환자 수'
        },
        
        "🧪 Testing (검사)": {
            'total_tests': '누적 검사 수',
            'new_tests': '일일 신규 검사 수',
            'total_tests_per_thousand': '인구 1000명 당 누적 검사 수',
            'new_tests_per_thousand': '인구 1000명 당 일일 신규 검사 수',
            'new_tests_smoothed': '일일 신규 검사 수 (7일 이동평균)',
            'new_tests_smoothed_per_thousand': '인구 1000명 당 일일 신규 검사 수 (7일 이동평균)',
            'positive_rate': '양성률 (확진자 수 / 검사 수)',
            'tests_per_case': '확진자 1명당 검사 수'
        },
        
        "📊 Epidemiological Indicators (역학 지표)": {
            'reproduction_rate': '재생산 지수 (R값) - 1명이 감염시키는 평균 인원수',
            'stringency_index': '방역 정책 엄격도 지수 (0-100, 높을수록 엄격)',
            'excess_mortality': '초과 사망률 (%)',
            'excess_mortality_cumulative': '누적 초과 사망률 (%)',
            'excess_mortality_cumulative_absolute': '누적 초과 사망자 수 (절댓값)',
            'excess_mortality_cumulative_per_million': '인구 100만명 당 누적 초과 사망자 수'
        },
        
        "🌍 Country Information (국가 정보)": {
            'country': '국가명',
            'code': '국가 코드 (ISO 3166-1 alpha-3)',
            'continent': '대륙',
            'population': '인구수',
            'population_density': '인구 밀도 (명/km²)',
            'median_age': '중위 연령',
            'life_expectancy': '기대 수명',
            'gdp_per_capita': '1인당 GDP (USD)',
            'extreme_poverty': '극빈층 비율 (%)',
            'diabetes_prevalence': '당뇨병 유병률 (%)',
            'handwashing_facilities': '손씻기 시설 접근성 (%)',
            'hospital_beds_per_thousand': '인구 1000명 당 병상 수',
            'human_development_index': '인간개발지수 (HDI, 0-1)'
        },
        
        "🔧 Metadata (메타데이터)": {
            'date': '날짜 (YYYY-MM-DD)',
            'collected_at': '데이터 수집 시각',
            'data_source': '데이터 출처',
            'collector': '수집 담당자'
        }
    }
    
    # 실제 데이터에 있는 컬럼들 분석
    actual_columns = set(data.columns)
    
    for group_name, columns in column_groups.items():
        print(f"\n{group_name}")
        print("-" * 50)
        
        group_found = False
        for col, description in columns.items():
            if col in actual_columns:
                group_found = True
                # 데이터 가용성 체크
                non_null = data[col].notna().sum()
                total = len(data)
                percentage = (non_null / total) * 100
                
                # 샘플 값
                sample_values = data[col].dropna().tail(3).values
                
                print(f"  ✅ {col}")
                print(f"     📝 {description}")
                print(f"     📊 데이터 가용성: {non_null:,}/{total:,} ({percentage:.1f}%)")
                
                if len(sample_values) > 0:
                    if col == 'date':
                        print(f"     📋 최근 값: {sample_values[-1] if len(sample_values) > 0 else 'N/A'}")
                    elif data[col].dtype in ['float64', 'int64']:
                        formatted_samples = [f"{v:,.0f}" if not pd.isna(v) else "N/A" for v in sample_values]
                        print(f"     📋 최근 값: {', '.join(formatted_samples)}")
                    else:
                        print(f"     📋 최근 값: {', '.join(map(str, sample_values))}")
                print()
        
        if not group_found:
            print(f"  ❌ 이 그룹의 컬럼들이 데이터에 없습니다.")
    
    # 분류되지 않은 컬럼들 체크
    classified_columns = set()
    for columns in column_groups.values():
        classified_columns.update(columns.keys())
    
    unclassified = actual_columns - classified_columns
    if unclassified:
        print(f"\n❓ 분류되지 않은 컬럼들:")
        print("-" * 50)
        for col in sorted(unclassified):
            print(f"  ? {col}")

# 분석 실행
analyze_column_info(korea_data)

📋 COVID-19 Dataset Column Information

📈 Cases & Deaths (확진자 & 사망자)
--------------------------------------------------
  ✅ total_cases
     📝 누적 확진자 수
     📊 데이터 가용성: 2,060/2,067 (99.7%)
     📋 최근 값: 34,571,873, 34,571,873, 34,571,873

  ✅ new_cases
     📝 일일 신규 확진자 수
     📊 데이터 가용성: 2,060/2,067 (99.7%)
     📋 최근 값: 0, 0, 0

  ✅ new_cases_smoothed
     📝 일일 신규 확진자 수 (7일 이동평균)
     📊 데이터 가용성: 2,055/2,067 (99.4%)
     📋 최근 값: 0, 0, 0

  ✅ total_cases_per_million
     📝 인구 100만명 당 누적 확진자 수
     📊 데이터 가용성: 2,060/2,067 (99.7%)
     📋 최근 값: 667,637, 667,637, 667,637

  ✅ new_cases_per_million
     📝 인구 100만명 당 일일 신규 확진자 수
     📊 데이터 가용성: 2,060/2,067 (99.7%)
     📋 최근 값: 0, 0, 0

  ✅ new_cases_smoothed_per_million
     📝 인구 100만명 당 일일 신규 확진자 수 (7일 이동평균)
     📊 데이터 가용성: 2,055/2,067 (99.4%)
     📋 최근 값: 0, 0, 0

  ✅ total_deaths
     📝 누적 사망자 수
     📊 데이터 가용성: 2,060/2,067 (99.7%)
     📋 최근 값: 35,934, 35,934, 35,934

  ✅ new_deaths
     📝 일일 신규 사망자 수
     📊 데이터 가용성: 2,059/2,067 (99.6%)
     📋 최근

In [20]:
# Cell 6-2: 모델링 관점에서 중요한 컬럼들 정리
print("\n🎯 모델링 관점에서 중요한 컬럼들")
print("=" * 60)

modeling_columns = {
    "🎯 Target Variables (예측 대상)": {
        'new_cases': '일일 신규 확진자 수 - 주요 예측 타겟',
        'new_cases_smoothed': '일일 신규 확진자 수 (7일 이동평균) - 노이즈 제거된 타겟',
        'new_deaths': '일일 신규 사망자 수 - 보조 예측 타겟'
    },
    
    "📊 Key Features (주요 피처)": {
        'people_vaccinated_per_hundred': '1차 접종률 - 감염률과 강한 역상관',
        'people_fully_vaccinated_per_hundred': '완전 접종률 - 중증도/사망률과 역상관',
        'stringency_index': '방역 정책 엄격도 - 사회적 거리두기 등 정책 효과',
        'reproduction_rate': '재생산 지수 - 전파력 직접 지표',
        'positive_rate': '양성률 - 실제 감염 확산 정도'
    },
    
    "📈 Time Series Features (시계열 피처)": {
        'date': '날짜 - 시간 정보 (요일, 계절성 등)',
        'total_cases': '누적 확진자 - 집단면역 수준 파악',
        'new_cases_smoothed': '평활화된 확진자 - 트렌드 파악'
    },
    
    "🏥 Healthcare Load (의료 부하)": {
        'hosp_patients_per_million': '입원 환자 비율 - 의료시스템 부하',
        'icu_patients_per_million': '중환자실 환자 비율 - 중증도 지표'
    },
    
    "🌍 Contextual Features (맥락 정보)": {
        'population_density': '인구 밀도 - 전파 속도에 영향',
        'median_age': '중위 연령 - 중증도에 영향',
        'gdp_per_capita': '경제 수준 - 방역 능력에 영향'
    }
}

print("박준영님을 위한 모델링 가이드:")
print()

for category, columns in modeling_columns.items():
    print(f"{category}")
    print("-" * 40)
    
    for col, description in columns.items():
        if col in korea_data.columns:
            availability = korea_data[col].notna().sum() / len(korea_data) * 100
            status = "🟢" if availability > 80 else "🟡" if availability > 50 else "🔴"
            
            print(f"  {status} {col}")
            print(f"     {description}")
            print(f"     데이터 가용성: {availability:.1f}%")
            
            # 상관관계 힌트 (new_cases와의 관계)
            if col != 'new_cases' and col in korea_data.columns and 'new_cases' in korea_data.columns:
                try:
                    correlation = korea_data[['new_cases', col]].corr().iloc[0, 1]
                    if not pd.isna(correlation):
                        corr_desc = "강한 " if abs(correlation) > 0.7 else "중간 " if abs(correlation) > 0.3 else "약한 "
                        corr_direction = "양의 " if correlation > 0 else "음의 "
                        print(f"     📈 신규 확진자와 {corr_desc}{corr_direction}상관관계 ({correlation:.3f})")
                except:
                    pass
            print()
        else:
            print(f"  ❌ {col} - 데이터에 없음")
            print(f"     {description}")
            print()

print("\n💡 모델링 제안사항:")
print("1. 🎯 주요 타겟: new_cases_smoothed (노이즈가 적어 예측하기 좋음)")
print("2. 📊 핵심 피처: 백신 접종률, 방역 정책 지수, 재생산 지수")
print("3. 📅 시계열 특성: 요일 효과, 계절성, 정책 변화 시점 고려")
print("4. 🔄 지연 효과: 정책이나 백신의 효과는 1-2주 지연될 수 있음")


🎯 모델링 관점에서 중요한 컬럼들
박준영님을 위한 모델링 가이드:

🎯 Target Variables (예측 대상)
----------------------------------------
  🟢 new_cases
     일일 신규 확진자 수 - 주요 예측 타겟
     데이터 가용성: 99.7%

  🟢 new_cases_smoothed
     일일 신규 확진자 수 (7일 이동평균) - 노이즈 제거된 타겟
     데이터 가용성: 99.4%
     📈 신규 확진자와 강한 양의 상관관계 (0.917)

  🟢 new_deaths
     일일 신규 사망자 수 - 보조 예측 타겟
     데이터 가용성: 99.6%
     📈 신규 확진자와 강한 양의 상관관계 (0.765)

📊 Key Features (주요 피처)
----------------------------------------
  🔴 people_vaccinated_per_hundred
     1차 접종률 - 감염률과 강한 역상관
     데이터 가용성: 31.7%
     📈 신규 확진자와 중간 양의 상관관계 (0.351)

  🔴 people_fully_vaccinated_per_hundred
     완전 접종률 - 중증도/사망률과 역상관
     데이터 가용성: 31.7%
     📈 신규 확진자와 중간 양의 상관관계 (0.390)

  🟡 stringency_index
     방역 정책 엄격도 - 사회적 거리두기 등 정책 효과
     데이터 가용성: 53.0%
     📈 신규 확진자와 약한 음의 상관관계 (-0.259)

  🟡 reproduction_rate
     재생산 지수 - 전파력 직접 지표
     데이터 가용성: 50.7%
     📈 신규 확진자와 약한 양의 상관관계 (0.032)

  🔴 positive_rate
     양성률 - 실제 감염 확산 정도
     데이터 가용성: 41.5%
     📈 신규 확진자와 강한 양의 상관관계 (0.770)

📈 Tim

In [21]:
# Cell 6-3: 데이터 검증을 위한 품질 체크리스트
print("\n🔍 이동건님을 위한 데이터 품질 체크리스트")
print("=" * 60)

quality_checks = {
    "📊 기본 무결성 체크": [
        "음수 값 체크 (확진자, 사망자, 백신 접종 등은 음수가 될 수 없음)",
        "날짜 순서 정렬 확인",
        "중복 날짜 확인",
        "누적 값의 단조증가 확인 (total_cases, total_deaths 등)"
    ],
    
    "📈 논리적 일관성 체크": [
        "new_cases와 total_cases의 일관성 (전날 total + new = 오늘 total)",
        "백신 접종률이 100%를 초과하지 않는지 확인",
        "people_fully_vaccinated <= people_vaccinated 확인",
        "사망률이 합리적 범위인지 확인 (new_deaths/new_cases)"
    ],
    
    "🚩 이상치 탐지 대상": [
        "급격한 확진자 증가/감소 (일일 변화율 300% 이상)",
        "사망자 수 급증 (평소의 10배 이상)",
        "백신 접종률 급변 (일일 변화 10% 이상)",
        "재생산 지수 극값 (R < 0.1 또는 R > 10)"
    ],
    
    "❓ 결측치 패턴 분석": [
        "백신 데이터: 2021년 이전 결측 (정상)",
        "검사 데이터: 국가별로 다른 시작 시점",
        "입원 데이터: 일부 국가만 제공",
        "연속된 결측치 구간 확인"
    ],
    
    "🔧 전처리 권장사항": [
        "7일 이동평균 데이터 우선 사용 (노이즈 제거)",
        "주말 효과 보정 (월요일 과소, 화요일 과다 보고)",
        "정책 변화 시점 전후 데이터 주의",
        "백신 접종 초기 데이터 급변 구간 평활화"
    ]
}

for category, items in quality_checks.items():
    print(f"\n{category}")
    print("-" * 40)
    for i, item in enumerate(items, 1):
        print(f"  {i}. {item}")

print(f"\n📋 권장 검증 순서:")
print("1. 기본 무결성 체크 → 2. 논리적 일관성 → 3. 이상치 탐지 → 4. 결측치 처리 → 5. 전처리")

# 간단한 품질 체크 예시
print(f"\n🔍 빠른 품질 체크 결과:")
print("-" * 30)

# 음수 값 체크
numeric_cols = ['new_cases', 'new_deaths', 'people_vaccinated_per_hundred']
for col in numeric_cols:
    if col in korea_data.columns:
        negative_count = (korea_data[col] < 0).sum()
        print(f"  {col} 음수 값: {negative_count}개 {'⚠️' if negative_count > 0 else '✅'}")

# 백신 접종률 100% 초과 체크
vaccine_cols = ['people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred']
for col in vaccine_cols:
    if col in korea_data.columns:
        over_100 = (korea_data[col] > 100).sum()
        print(f"  {col} 100% 초과: {over_100}개 {'⚠️' if over_100 > 0 else '✅'}")

# 결측치 연속성 체크
if 'new_cases' in korea_data.columns:
    consecutive_na = korea_data['new_cases'].isna().sum()
    print(f"  new_cases 결측치: {consecutive_na}개 {'⚠️' if consecutive_na > 100 else '✅'}")


🔍 이동건님을 위한 데이터 품질 체크리스트

📊 기본 무결성 체크
----------------------------------------
  1. 음수 값 체크 (확진자, 사망자, 백신 접종 등은 음수가 될 수 없음)
  2. 날짜 순서 정렬 확인
  3. 중복 날짜 확인
  4. 누적 값의 단조증가 확인 (total_cases, total_deaths 등)

📈 논리적 일관성 체크
----------------------------------------
  1. new_cases와 total_cases의 일관성 (전날 total + new = 오늘 total)
  2. 백신 접종률이 100%를 초과하지 않는지 확인
  3. people_fully_vaccinated <= people_vaccinated 확인
  4. 사망률이 합리적 범위인지 확인 (new_deaths/new_cases)

🚩 이상치 탐지 대상
----------------------------------------
  1. 급격한 확진자 증가/감소 (일일 변화율 300% 이상)
  2. 사망자 수 급증 (평소의 10배 이상)
  3. 백신 접종률 급변 (일일 변화 10% 이상)
  4. 재생산 지수 극값 (R < 0.1 또는 R > 10)

❓ 결측치 패턴 분석
----------------------------------------
  1. 백신 데이터: 2021년 이전 결측 (정상)
  2. 검사 데이터: 국가별로 다른 시작 시점
  3. 입원 데이터: 일부 국가만 제공
  4. 연속된 결측치 구간 확인

🔧 전처리 권장사항
----------------------------------------
  1. 7일 이동평균 데이터 우선 사용 (노이즈 제거)
  2. 주말 효과 보정 (월요일 과소, 화요일 과다 보고)
  3. 정책 변화 시점 전후 데이터 주의
  4. 백신 접종 초기 데이터 급변 구간 평활화

📋 권장 검증 순서:
1. 기본 무결성 체크 → 2. 논리적 일관성 → 3. 이

In [14]:
# Cell 7 (수정): 개선된 수집기 클래스 - 실제 데이터만 수집
import os
from datetime import datetime, date
import pandas as pd

class RawCovidCollector:
    def __init__(self):
        self.owid_url = "https://catalog.ourworldindata.org/garden/covid/latest/compact/compact.csv"
        self.country_column = 'country'
        self.korea_name = 'South Korea'
        
    def collect_raw_data(self, filter_future_dates=True):
        """원시 데이터 수집 - 가공하지 않음"""
        print("📥 Collecting raw COVID data...")
        
        # 청크 단위로 읽어서 한국 데이터만 추출
        korea_chunks = []
        chunk_size = 10000
        chunks_processed = 0
        
        print(f"🔍 Searching for Korea data in chunks...")
        
        for chunk in pd.read_csv(self.owid_url, chunksize=chunk_size):
            chunks_processed += 1
            
            # 한국 데이터가 있는 청크 찾기
            korea_in_chunk = chunk[chunk[self.country_column] == self.korea_name].copy()
            
            if len(korea_in_chunk) > 0:
                korea_chunks.append(korea_in_chunk)
                print(f"✅ Found Korea data in chunk {chunks_processed}: {len(korea_in_chunk)} rows")
                
                # 한국 데이터를 모두 찾았으면 중단
                if len(korea_in_chunk) < chunk_size:
                    break
        
        # 모든 한국 데이터 청크 결합
        if korea_chunks:
            korea_raw = pd.concat(korea_chunks, ignore_index=True)
            print(f"🇰🇷 Total Korea data collected: {len(korea_raw)} rows")
        else:
            raise ValueError("❌ No Korea data found in the dataset")
        
        # 날짜 처리
        korea_raw['date'] = pd.to_datetime(korea_raw['date'])
        
        # 미래 날짜 필터링 (옵션)
        if filter_future_dates:
            today = date.today()
            before_filter = len(korea_raw)
            korea_raw = korea_raw[korea_raw['date'].dt.date <= today].copy()
            after_filter = len(korea_raw)
            print(f"🗓️  Filtered future dates: {before_filter} → {after_filter} rows (removed {before_filter - after_filter} future dates)")
        
        # 수집 메타데이터 추가
        korea_raw['collected_at'] = datetime.now()
        korea_raw['data_source'] = 'OWID'
        korea_raw['collector'] = '최해혁'
        
        return korea_raw
    
    def save_raw_data(self, data):
        """원시 데이터 그대로 저장"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # 디렉토리 생성
        os.makedirs('./data/raw', exist_ok=True)
        
        filename = f"./data/raw/korea_covid_raw_{timestamp}.csv"
        
        # 원본 그대로 저장 (결측치, 이상치 포함)
        data.to_csv(filename, index=False)
        
        print(f"💾 Raw data saved: {filename}")
        print(f"📊 Rows: {len(data):,}, Columns: {len(data.columns)}")
        print(f"📅 Date range: {data['date'].min()} ~ {data['date'].max()}")
        
        # 파일 크기 확인
        file_size = os.path.getsize(filename) / 1024 / 1024
        print(f"📁 File size: {file_size:.2f} MB")
        
        # 데이터 품질 요약
        print(f"📋 Data quality summary:")
        missing_cols = data.isnull().any()
        print(f"  📊 Columns with missing data: {missing_cols.sum()}/{len(data.columns)}")
        print(f"  📊 Complete rows: {len(data.dropna())}/{len(data)} ({len(data.dropna())/len(data)*100:.1f}%)")
        
        return filename
    
    def basic_data_info(self, data):
        """기본 데이터 정보만 제공 (품질 분석은 안함)"""
        info = {
            'total_rows': len(data),
            'date_range': (data['date'].min(), data['date'].max()) if 'date' in data.columns else (None, None),
            'columns': list(data.columns),
            'null_counts': data.isnull().sum().to_dict(),
            'country_name': self.korea_name,
            'data_source': 'Our World in Data (OWID)',
            'file_size_mb': data.memory_usage(deep=True).sum() / 1024**2,
            'collection_notes': {
                'missing_vaccine_data': 'Early 2020-2021 period (before vaccination campaign)',
                'data_quality': 'High for cases/deaths, moderate for vaccines',
                'future_dates_filtered': True
            }
        }
        return info

print("✅ Enhanced RawCovidCollector class defined!")
print("🔧 Improvements:")
print("  - Fixed SettingWithCopyWarning")
print("  - Filters out future dates automatically")
print("  - Enhanced metadata and quality reporting")
print("  - Better error handling")

✅ Enhanced RawCovidCollector class defined!
🔧 Improvements:
  - Fixed SettingWithCopyWarning
  - Filters out future dates automatically
  - Enhanced metadata and quality reporting
  - Better error handling


In [15]:
# Cell 8 (수정): 개선된 테스트
print("🧪 Testing enhanced collector with cleaned Korea data...")

try:
    # 수집기 인스턴스 생성
    collector = RawCovidCollector()
    print(f"✅ Enhanced collector created for country: {collector.korea_name}")
    
    # 정제된 korea_data 사용 (미래 날짜 제거됨)
    print("📊 Testing with cleaned Korea data...")
    data_info = collector.basic_data_info(korea_data)
    
    print("✅ Data info generated successfully!")
    print(f"  📊 Total rows: {data_info['total_rows']:,}")
    print(f"  📅 Date range: {data_info['date_range'][0].strftime('%Y-%m-%d')} ~ {data_info['date_range'][1].strftime('%Y-%m-%d')}")
    print(f"  🏷️ Country: {data_info['country_name']}")
    print(f"  💾 Memory usage: {data_info['file_size_mb']:.2f} MB")
    print(f"  🔗 Source: {data_info['data_source']}")
    
    # 파일 저장 테스트
    print(f"\n💾 Testing enhanced file save...")
    file_path = collector.save_raw_data(korea_data)
    
    # 저장된 파일 검증
    print(f"\n🔍 Verifying saved file...")
    saved_data = pd.read_csv(file_path)
    saved_data['date'] = pd.to_datetime(saved_data['date'])
    
    print(f"✅ File verification successful!")
    print(f"  📁 Saved file: {file_path}")
    print(f"  📊 Loaded rows: {len(saved_data):,}")
    print(f"  📅 Date range in file: {saved_data['date'].min()} ~ {saved_data['date'].max()}")
    print(f"  🔍 Data integrity: {'✅ PASS' if len(saved_data) == len(korea_data) else '❌ FAIL'}")
    
except Exception as e:
    print(f"❌ Test failed: {e}")
    import traceback
    traceback.print_exc()

🧪 Testing enhanced collector with cleaned Korea data...
✅ Enhanced collector created for country: South Korea
📊 Testing with cleaned Korea data...
✅ Data info generated successfully!
  📊 Total rows: 2,067
  📅 Date range: 2020-01-01 ~ 2025-09-21
  🏷️ Country: South Korea
  💾 Memory usage: 1.30 MB
  🔗 Source: Our World in Data (OWID)

💾 Testing enhanced file save...
💾 Raw data saved: ./data/raw/korea_covid_raw_20250923_025616.csv
📊 Rows: 2,067, Columns: 61
📅 Date range: 2020-01-01 00:00:00 ~ 2025-09-21 00:00:00
📁 File size: 0.72 MB
📋 Data quality summary:
  📊 Columns with missing data: 49/61
  📊 Complete rows: 0/2067 (0.0%)

🔍 Verifying saved file...
✅ File verification successful!
  📁 Saved file: ./data/raw/korea_covid_raw_20250923_025616.csv
  📊 Loaded rows: 2,067
  📅 Date range in file: 2020-01-01 00:00:00 ~ 2025-09-21 00:00:00
  🔍 Data integrity: ✅ PASS


In [16]:
# Cell 9: 실제 데이터 수집 테스트 (새로 다운로드)
print("🚀 Testing full data collection process...")

try:
    # 새로운 수집기 인스턴스로 실제 데이터 수집
    fresh_collector = RawCovidCollector()
    
    print("⚠️  This will download data again - it may take a moment...")
    user_input = input("Continue? (y/n): ")
    
    if user_input.lower() == 'y':
        # 실제 데이터 수집 실행
        fresh_korea_data = fresh_collector.collect_raw_data()
        
        # 수집 결과 확인
        fresh_data_info = fresh_collector.basic_data_info(fresh_korea_data)
        print("✅ Fresh data collection successful!")
        
        # 결과 저장
        fresh_file_path = fresh_collector.save_raw_data(fresh_korea_data)
        
        print(f"\n🎉 Collection completed!")
        print(f"📁 New file saved: {fresh_file_path}")
        print(f"📊 Fresh data: {len(fresh_korea_data)} rows")
        
    else:
        print("⏭️  Skipped fresh data collection")
        fresh_file_path = file_path  # 이전 파일 사용
        
except Exception as e:
    print(f"❌ Fresh collection failed: {e}")
    fresh_file_path = file_path  # 이전 파일 사용

🚀 Testing full data collection process...
⚠️  This will download data again - it may take a moment...


Continue? (y/n):  y


📥 Collecting raw COVID data...
🔍 Searching for Korea data in chunks...
✅ Found Korea data in chunk 44: 2081 rows
🇰🇷 Total Korea data collected: 2081 rows
🗓️  Filtered future dates: 2081 → 2067 rows (removed 14 future dates)
✅ Fresh data collection successful!
💾 Raw data saved: ./data/raw/korea_covid_raw_20250923_025636.csv
📊 Rows: 2,067, Columns: 64
📅 Date range: 2020-01-01 00:00:00 ~ 2025-09-21 00:00:00
📁 File size: 0.80 MB
📋 Data quality summary:
  📊 Columns with missing data: 49/64
  📊 Complete rows: 0/2067 (0.0%)

🎉 Collection completed!
📁 New file saved: ./data/raw/korea_covid_raw_20250923_025636.csv
📊 Fresh data: 2067 rows


In [18]:
# Cell 10: 최종 요약 및 다음 단계
print("🎉 Data Collection Phase Completed!")
print("=" * 60)

# 최종 결과 요약
final_data = pd.read_csv(fresh_file_path if 'fresh_file_path' in locals() else file_path)

print(f"✅ Data collection successful!")
print(f"📁 Final file: {fresh_file_path if 'fresh_file_path' in locals() else file_path}")
print(f"📊 Total rows: {len(final_data):,}")
print(f"📊 Total columns: {len(final_data.columns)}")
print(f"📅 Date coverage: {final_data['date'].min()} to {final_data['date'].max()}")

# 데이터 품질 기본 체크
print(f"\n📋 Basic data quality overview:")
print(f"  🔍 Missing values detected in {final_data.isnull().any().sum()} columns")
print(f"  📊 Complete rows: {len(final_data.dropna())}/{len(final_data)} ({len(final_data.dropna())/len(final_data)*100:.1f}%)")

# 이동건님에게 전달할 정보
handoff_info = {
    'collector_name': '최해혁',
    'collection_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'raw_data_file': fresh_file_path if 'fresh_file_path' in locals() else file_path,
    'country': 'South Korea',
    'data_source': 'Our World in Data (OWID)',
    'total_rows': len(final_data),
    'date_range': (final_data['date'].min(), final_data['date'].max()),
    'key_columns': [
        'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
        'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred'
    ],
    'notes': 'Raw data includes missing values for early vaccination period (expected)'
}

print(f"\n📋 Information for 이동건님 (Data Validator):")
print("=" * 50)
for key, value in handoff_info.items():
    print(f"  {key}: {value}")

print(f"\n👉 Next Steps:")
print(f"1. ✅ 데이터 수집 완료 (최해혁)")
print(f"2. 🔄 데이터 검증 시작 (이동건님)")
print(f"3. ⏳ 피처 엔지니어링 대기 (박준영님)")
print(f"4. ⏳ 모델 개발 대기 (박준영님)")

print(f"\n🎯 Ready for next phase!")

🎉 Data Collection Phase Completed!
✅ Data collection successful!
📁 Final file: ./data/raw/korea_covid_raw_20250923_025636.csv
📊 Total rows: 2,067
📊 Total columns: 64
📅 Date coverage: 2020-01-01 to 2025-09-21

📋 Basic data quality overview:
  🔍 Missing values detected in 49 columns
  📊 Complete rows: 0/2067 (0.0%)

📋 Information for 이동건님 (Data Validator):
  collector_name: 최해혁
  collection_date: 2025-09-23 02:57:41
  raw_data_file: ./data/raw/korea_covid_raw_20250923_025636.csv
  country: South Korea
  data_source: Our World in Data (OWID)
  total_rows: 2067
  date_range: ('2020-01-01', '2025-09-21')
  key_columns: ['date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred']
  notes: Raw data includes missing values for early vaccination period (expected)

👉 Next Steps:
1. ✅ 데이터 수집 완료 (최해혁)
2. 🔄 데이터 검증 시작 (이동건님)
3. ⏳ 피처 엔지니어링 대기 (박준영님)
4. ⏳ 모델 개발 대기 (박준영님)

🎯 Ready for next phase!
